# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,chaihe,44.7667,129.7000,-25.72,99,35,0.45,CN,1707074150
1,1,ghat,24.9647,10.1728,13.04,24,0,3.12,LY,1707073553
2,2,mumford,5.2618,-0.7590,26.97,80,63,4.61,GH,1707074150
3,3,ribeira grande,38.5167,-28.7000,17.65,94,75,9.39,PT,1707074150
4,4,porto novo,6.4965,2.6036,26.10,73,99,2.89,BJ,1707074150


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True, 
    color = "City",
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500, 
    alpha = 0.6,
    size = "Humidity",
    scale = 0.8
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[(city_data_df['Max Temp'] <27)
                        &(city_data_df['Max Temp'] >21)
                        &(city_data_df['Cloudiness'] == 0)
                        &(city_data_df['Wind Speed'] <=4.5),:]

# Drop any rows with null values
ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
88,88,quila,24.4228,-107.2200,23.92,20,0,3.51,MX,1707074163
150,150,salalah,17.0151,54.0924,25.05,61,0,1.54,OM,1707074178
174,174,nova sintra,14.8667,-24.7167,21.52,69,0,3.85,CV,1707074182
280,280,zayed city,24.3768,54.5890,21.67,54,0,3.09,AE,1707074199
296,296,murud,18.3258,72.9650,24.04,64,0,1.83,IN,1707074202
324,324,isabela,6.7085,121.9711,25.51,84,0,1.67,PH,1707074206
441,441,pacocha,-17.5833,-71.3333,25.76,70,0,3.15,PE,1707074224


### Step 3: Create a new DataFrame called `hotel_df`.

In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
88,quila,MX,24.4228,-107.2200,20,
150,salalah,OM,17.0151,54.0924,61,
174,nova sintra,CV,14.8667,-24.7167,69,
280,zayed city,AE,24.3768,54.5890,54,
296,murud,IN,18.3258,72.9650,64,
324,isabela,PH,6.7085,121.9711,84,
441,pacocha,PE,-17.5833,-71.3333,70,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
          "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
quila - nearest hotel: No hotel found
salalah - nearest hotel: Muscat International Hotel
nova sintra - nearest hotel: Residência Ka Dencho
zayed city - nearest hotel: L'Arabia hotel
murud - nearest hotel: No hotel found
isabela - nearest hotel: No hotel found
pacocha - nearest hotel: Gran Hotel Ilo


,City,Country,Lat,Lng,Humidity,Hotel Name
88,quila,MX,24.4228,-107.2200,20,No hotel found
150,salalah,OM,17.0151,54.0924,61,Muscat International Hotel
174,nova sintra,CV,14.8667,-24.7167,69,Residência Ka Dencho
280,zayed city,AE,24.3768,54.5890,54,L'Arabia hotel
296,murud,IN,18.3258,72.9650,64,No hotel found
324,isabela,PH,6.7085,121.9711,84,No hotel found
441,pacocha,PE,-17.5833,-71.3333,70,Gran Hotel Ilo


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [26]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True, 
    color = "City",
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500, 
    alpha = 0.6,
    size = "Humidity",
    scale = 1.2,
    hover_cols= ["Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)